<a href="https://www.kaggle.com/code/rhythmshah11/titanic-rf-kaggle?scriptVersionId=170837465" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
df_train=pd.read_csv('/kaggle/input/titanic/train.csv')
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
df_test=pd.read_csv('/kaggle/input/titanic/test.csv')

In [4]:
df_train.drop(['Name'],axis=1,inplace=True)
df_test.drop(['Name'],axis=1,inplace=True)
df_train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,male,26.0,0,0,111369,30.0000,C148,C


In [5]:
df_train['Sex'].unique()

array(['male', 'female'], dtype=object)

In [6]:
df_train['Sex'] = df_train['Sex'].map({'male': 1, 'female': 0})
df_test['Sex'] = df_test['Sex'].map({'male': 1, 'female': 0})


In [7]:
null_percentage = round((df_train.isnull().sum() / len(df_train)) * 100,2)
null_percentage.sort_values(ascending=False)[:3]

Cabin       77.10
Age         19.87
Embarked     0.22
dtype: float64

In [8]:
df_train['Cabin'].isnull().sum()

687

In [9]:
df_train[df_train.isnull().any(axis=1)]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,1,22.0,1,0,A/5 21171,7.2500,NaN,S
2,3,1,3,0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,5,0,3,1,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,1,NaN,0,0,330877,8.4583,NaN,Q
7,8,0,3,1,2.0,3,1,349909,21.0750,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
884,885,0,3,1,25.0,0,0,SOTON/OQ 392076,7.0500,NaN,S
885,886,0,3,0,39.0,0,5,382652,29.1250,NaN,Q
886,887,0,2,1,27.0,0,0,211536,13.0000,NaN,S
888,889,0,3,0,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [10]:
df_train.drop(['PassengerId','Cabin'],axis=1,inplace=True)
df_test.drop(['Cabin'],axis=1,inplace=True)
df_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,0,3,1,22.0,1,0,A/5 21171,7.2500,S
1,1,1,0,38.0,1,0,PC 17599,71.2833,C
2,1,3,0,26.0,0,0,STON/O2. 3101282,7.9250,S
3,1,1,0,35.0,1,0,113803,53.1000,S
4,0,3,1,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,211536,13.0000,S
887,1,1,0,19.0,0,0,112053,30.0000,S
888,0,3,0,NaN,1,2,W./C. 6607,23.4500,S
889,1,1,1,26.0,0,0,111369,30.0000,C


In [11]:
df_train['Age'].dtype

dtype('float64')

In [12]:
df_train['Age'].fillna(round(df_train['Age'].mean()),inplace=True)
df_test['Age'].fillna(round(df_test['Age'].mean()),inplace=True)

/tmp/ipykernel_33/2896906292.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train['Age'].fillna(round(df_train['Age'].mean()),inplace=True)
/tmp/ipykernel_33/2896906292.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplac

In [13]:
df_train['Age'] = df_train['Age'].astype('int')
df_test['Age'] = df_test['Age'].astype('int')
df_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,0,3,1,22,1,0,A/5 21171,7.2500,S
1,1,1,0,38,1,0,PC 17599,71.2833,C
2,1,3,0,26,0,0,STON/O2. 3101282,7.9250,S
3,1,1,0,35,1,0,113803,53.1000,S
4,0,3,1,35,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...
886,0,2,1,27,0,0,211536,13.0000,S
887,1,1,0,19,0,0,112053,30.0000,S
888,0,3,0,30,1,2,W./C. 6607,23.4500,S
889,1,1,1,26,0,0,111369,30.0000,C


In [14]:
df_test[df_test['Fare'].isna()]

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
152,1044,3,1,60,0,0,3701,NaN,S


In [15]:
df_test['Fare'].fillna(round(df_test['Fare'].mean(),3),inplace=True)

/tmp/ipykernel_33/2751532610.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_test['Fare'].fillna(round(df_test['Fare'].mean(),3),inplace=True)


In [16]:
df_test[df_test['Fare'].isna()]

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked


In [17]:
null_percentage = round((df_train.isnull().sum() / len(df_train)) * 100,2)
null_percentage.sort_values(ascending=False)[:3]

Embarked    0.22
Survived    0.00
Pclass      0.00
dtype: float64

In [18]:
df_train.dropna(subset=['Embarked'],inplace=True)

In [19]:
null_percentage = round((df_train.isnull().sum() / len(df_train)) * 100,2)
null_percentage.sort_values(ascending=False)[:3]

Survived    0.0
Pclass      0.0
Sex         0.0
dtype: float64

In [20]:
df_train.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
count,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000
mean,0.382452,2.311586,0.649044,29.697413,0.524184,0.382452,32.096681
std,0.486260,0.834700,0.477538,12.978135,1.103705,0.806761,49.697504
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,22.000000,0.000000,0.000000,7.895800
50%,0.000000,3.000000,1.000000,30.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,1.000000,35.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200


In [21]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 889 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  889 non-null    int64  
 1   Pclass    889 non-null    int64  
 2   Sex       889 non-null    int64  
 3   Age       889 non-null    int64  
 4   SibSp     889 non-null    int64  
 5   Parch     889 non-null    int64  
 6   Ticket    889 non-null    object 
 7   Fare      889 non-null    float64
 8   Embarked  889 non-null    object 
dtypes: float64(1), int64(6), object(2)
memory usage: 69.5+ KB


In [22]:
df_train['Ticket'].nunique()

680

In [23]:
df_train.drop(['Ticket'],axis=1,inplace=True)
df_test.drop(['Ticket'],axis=1,inplace=True)
df_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22,1,0,7.2500,S
1,1,1,0,38,1,0,71.2833,C
2,1,3,0,26,0,0,7.9250,S
3,1,1,0,35,1,0,53.1000,S
4,0,3,1,35,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,1,27,0,0,13.0000,S
887,1,1,0,19,0,0,30.0000,S
888,0,3,0,30,1,2,23.4500,S
889,1,1,1,26,0,0,30.0000,C


In [24]:
from sklearn.preprocessing import LabelEncoder

In [25]:
label_encoder = LabelEncoder()
df_train['Embarked'] = label_encoder.fit_transform(df_train['Embarked'])
df_test['Embarked'] = label_encoder.fit_transform(df_test['Embarked'])
df_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22,1,0,7.2500,2
1,1,1,0,38,1,0,71.2833,0
2,1,3,0,26,0,0,7.9250,2
3,1,1,0,35,1,0,53.1000,2
4,0,3,1,35,0,0,8.0500,2
...,...,...,...,...,...,...,...,...
886,0,2,1,27,0,0,13.0000,2
887,1,1,0,19,0,0,30.0000,2
888,0,3,0,30,1,2,23.4500,2
889,1,1,1,26,0,0,30.0000,0


In [26]:
df_test.isna().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [27]:
df_test.isna().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [28]:
df_test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34,0,0,7.8292,1
1,893,3,0,47,1,0,7.0000,2
2,894,2,1,62,0,0,9.6875,1
3,895,3,1,27,0,0,8.6625,2
4,896,3,0,22,1,1,12.2875,2
...,...,...,...,...,...,...,...,...
413,1305,3,1,30,0,0,8.0500,2
414,1306,1,0,39,0,0,108.9000,0
415,1307,3,1,38,0,0,7.2500,2
416,1308,3,1,30,0,0,8.0500,2


In [29]:
from sklearn.ensemble import RandomForestClassifier

In [30]:
rf_classifier = RandomForestClassifier(n_estimators=100,random_state=1)

In [31]:
x_train = df_train[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
x_test = df_test[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
x_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22,1,0,7.2500,2
1,1,0,38,1,0,71.2833,0
2,3,0,26,0,0,7.9250,2
3,1,0,35,1,0,53.1000,2
4,3,1,35,0,0,8.0500,2
...,...,...,...,...,...,...,...
886,2,1,27,0,0,13.0000,2
887,1,0,19,0,0,30.0000,2
888,3,0,30,1,2,23.4500,2
889,1,1,26,0,0,30.0000,0


In [32]:
y_train = df_train['Survived']
y_train

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 889, dtype: int64

In [33]:
rf_classifier.fit(x_train,y_train)

RandomForestClassifier(random_state=1)

In [34]:
y_test = rf_classifier.predict(x_test)
y_test

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [35]:
output = pd.DataFrame({'PassengerId': df_test.PassengerId, 'Survived': y_test})
output.to_csv('submission.csv', index=False)

In [36]:
output

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
